# Homework 5

# Bonus Question, (b)

In this section the IterGen was added steering refusal_direction which is by using that steering vector defined by two datasets that will lead to both desirable and undesirable behaviors, which helps to reduce the number of resampling. 

A few hacks are used to properly install/import as the pypi installer (setup.py) version will cause some files not installed for mxeval.

In [1]:
%%capture
!pip install torch transformers accelerate sentencepiece protobuf
!pip install matplotlib numpy tqdm einops jaxtyping
!pip install fire interegular datasets

In [2]:
%pip uninstall -y mxeval
!mkdir -p ./tmp
!rm -rf ./tmp/mxeval_repo
!git clone https://github.com/amazon-science/mxeval.git ./tmp/mxeval_repo
%pip install -e ./tmp/mxeval_repo

Found existing installation: mxeval 1.0
Uninstalling mxeval-1.0:
  Successfully uninstalled mxeval-1.0
Note: you may need to restart the kernel to use updated packages.
Cloning into './tmp/mxeval_repo'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 362 (delta 47), reused 41 (delta 41), pack-reused 301 (from 1)
Receiving objects: 100% (362/362), 9.11 MiB | 22.22 MiB/s, done.
Resolving deltas: 100% (161/161), done.
Obtaining file:///home/zhong42/tmp/mxeval_repo
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mxeval
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import sys
import subprocess
from pathlib import Path
os.environ["TOKENIZERS_PARALLELISM"] = "false"
TMP = Path("./tmp").resolve()
REPO = TMP / "itergen"

TMP.mkdir(parents=True, exist_ok=True)

if not REPO.exists():
    subprocess.check_call([
        "git", "clone", "--recursive",
        "https://github.com/structuredllm/itergen.git",
        str(REPO)
    ])

sys.path.insert(0, str(REPO))

import itergen
from itergen import main, trace

print("itergen loaded from:", itergen.__file__)


itergen loaded from: /home/zhong42/tmp/itergen/itergen/__init__.py


In [4]:
import os, sys, pathlib, importlib.util

print("CWD:", os.getcwd())
print("sys.path[0:5]:", sys.path[:5])

spec = importlib.util.find_spec("mxeval")
print("\nfind_spec('mxeval'):", spec)
if spec:
    print("  origin:", spec.origin)
    print("  loader:", spec.loader)
    print("  submodule_search_locations:", list(spec.submodule_search_locations or []))

spec2 = importlib.util.find_spec("mxeval.data")
print("\nfind_spec('mxeval.data'):", spec2)
if spec2:
    print("  origin:", spec2.origin)
    print("  loader:", spec2.loader)
    print("  submodule_search_locations:", list(spec2.submodule_search_locations or []))

print("\nPotential shadowing paths that contain a top-level 'mxeval':")
for p in sys.path:
    try:
        pp = pathlib.Path(p)
        if (pp / "mxeval").exists():
            print("  -", pp / "mxeval")
    except Exception:
        pass


CWD: /home/zhong42
sys.path[0:5]: ['/home/zhong42/tmp/itergen', '/home/zhong42', '/opt/conda/lib/python311.zip', '/opt/conda/lib/python3.11', '/opt/conda/lib/python3.11/lib-dynload']

find_spec('mxeval'): ModuleSpec(name='mxeval', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f3384d93a50>, origin='/home/zhong42/tmp/mxeval_repo/mxeval/__init__.py', submodule_search_locations=['/home/zhong42/tmp/mxeval_repo/mxeval'])
  origin: /home/zhong42/tmp/mxeval_repo/mxeval/__init__.py
  loader: <_frozen_importlib_external.SourceFileLoader object at 0x7f3384d93a50>
  submodule_search_locations: ['/home/zhong42/tmp/mxeval_repo/mxeval']

find_spec('mxeval.data'): ModuleSpec(name='mxeval.data', loader=<_frozen_importlib_external.SourceFileLoader object at 0x7f3373e47d50>, origin='/home/zhong42/tmp/mxeval_repo/mxeval/data.py')
  origin: /home/zhong42/tmp/mxeval_repo/mxeval/data.py
  loader: <_frozen_importlib_external.SourceFileLoader object at 0x7f3373e47d50>
  submodule_search_loca

In [5]:
import mxeval
import os
print(os.path.exists(os.path.dirname(mxeval.__file__) + '/data/multilingual_humaneval/metadata.json'))

print(os.path.dirname(mxeval.__file__))

False
/home/zhong42/tmp/mxeval_repo/mxeval


In [6]:
import torch
import random
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
from typing import List, Callable, Tuple
import contextlib
import functools
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig

# seed 42
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

## Dependencies

The part of code copied from andyrdt/refusal_direction, so we don't need to import from their repo. Since we only do gemma stuff from other models not included here.

In [7]:
from abc import ABC, abstractmethod
import torch
from torch import Tensor
import functools
from typing import List, Tuple, Callable
from jaxtyping import Float
from contextlib import contextmanager
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
from tqdm import tqdm
import einops
import os

GEMMA_CHAT_TEMPLATE = """<start_of_turn>user
{instruction}<end_of_turn>
<start_of_turn>model
"""

GEMMA_REFUSAL_TOKS = [235285] # ['I']

@contextmanager
def add_hooks(
    module_forward_pre_hooks: List[Tuple[torch.nn.Module, Callable]],
    module_forward_hooks: List[Tuple[torch.nn.Module, Callable]],
    **kwargs
):
    """
    Context manager for temporarily adding forward hooks to a model.

    Parameters
    ----------
    module_forward_pre_hooks
        A list of pairs: (module, fnc) The function will be registered as a
            forward pre hook on the module
    module_forward_hooks
        A list of pairs: (module, fnc) The function will be registered as a
            forward hook on the module
    """
    try:
        handles = []
        for module, hook in module_forward_pre_hooks:
            partial_hook = functools.partial(hook, **kwargs)
            handles.append(module.register_forward_pre_hook(partial_hook))
        for module, hook in module_forward_hooks:
            partial_hook = functools.partial(hook, **kwargs)
            handles.append(module.register_forward_hook(partial_hook))
        yield
    finally:
        for h in handles:
            h.remove()

def get_activation_addition_input_pre_hook(vector: Float[Tensor, "d_model"], coeff: Float[Tensor, ""]):
    def hook_fn(module, input):
        nonlocal vector

        if isinstance(input, tuple):
            activation: Float[Tensor, "batch_size seq_len d_model"] = input[0]
        else:
            activation: Float[Tensor, "batch_size seq_len d_model"] = input

        vector = vector.to(activation)
        activation += coeff * vector

        if isinstance(input, tuple):
            return (activation, *input[1:])
        else:
            return activation
    return hook_fn

def get_all_direction_ablation_hooks(
    model_base,
    direction: Float[Tensor, 'd_model'],
):
    fwd_pre_hooks = [(model_base.model_block_modules[layer], get_direction_ablation_input_pre_hook(direction=direction)) for layer in range(model_base.model.config.num_hidden_layers)]
    fwd_hooks = [(model_base.model_attn_modules[layer], get_direction_ablation_output_hook(direction=direction)) for layer in range(model_base.model.config.num_hidden_layers)]
    fwd_hooks += [(model_base.model_mlp_modules[layer], get_direction_ablation_output_hook(direction=direction)) for layer in range(model_base.model.config.num_hidden_layers)]

    return fwd_pre_hooks, fwd_hooks

def get_direction_ablation_input_pre_hook(direction: Tensor):
    def hook_fn(module, input):
        nonlocal direction

        if isinstance(input, tuple):
            activation: Float[Tensor, "batch_size seq_len d_model"] = input[0]
        else:
            activation: Float[Tensor, "batch_size seq_len d_model"] = input

        direction = direction / (direction.norm(dim=-1, keepdim=True) + 1e-8)
        direction = direction.to(activation) 
        activation -= (activation @ direction).unsqueeze(-1) * direction 

        if isinstance(input, tuple):
            return (activation, *input[1:])
        else:
            return activation
    return hook_fn

def get_direction_ablation_output_hook(direction: Tensor):
    def hook_fn(module, input, output):
        nonlocal direction

        if isinstance(output, tuple):
            activation: Float[Tensor, "batch_size seq_len d_model"] = output[0]
        else:
            activation: Float[Tensor, "batch_size seq_len d_model"] = output

        direction = direction / (direction.norm(dim=-1, keepdim=True) + 1e-8)
        direction = direction.to(activation)
        activation -= (activation @ direction).unsqueeze(-1) * direction 

        if isinstance(output, tuple):
            return (activation, *output[1:])
        else:
            return activation

    return hook_fn

class ModelBase(ABC):
    def __init__(self, model_name_or_path: str):
        self.model_name_or_path = model_name_or_path
        self.model: AutoModelForCausalLM = self._load_model(model_name_or_path)
        self.tokenizer: AutoTokenizer = self._load_tokenizer(model_name_or_path)
        
        self.tokenize_instructions_fn = self._get_tokenize_instructions_fn()
        self.eoi_toks = self._get_eoi_toks()
        self.refusal_toks = self._get_refusal_toks()

        self.model_block_modules = self._get_model_block_modules()
        self.model_attn_modules = self._get_attn_modules()
        self.model_mlp_modules = self._get_mlp_modules()

    def del_model(self):
        if hasattr(self, 'model') and self.model is not None:
            del self.model

    @abstractmethod
    def _load_model(self, model_name_or_path: str) -> AutoModelForCausalLM:
        pass

    @abstractmethod
    def _load_tokenizer(self, model_name_or_path: str) -> AutoTokenizer:
        pass

    @abstractmethod
    def _get_tokenize_instructions_fn(self):
        pass

    @abstractmethod
    def _get_eoi_toks(self):
        pass

    @abstractmethod
    def _get_refusal_toks(self):
        pass

    @abstractmethod
    def _get_model_block_modules(self):
        pass

    @abstractmethod
    def _get_attn_modules(self):
        pass

    @abstractmethod
    def _get_mlp_modules(self):
        pass

    @abstractmethod
    def _get_orthogonalization_mod_fn(self, direction: Float[Tensor, "d_model"]):
        pass

    @abstractmethod
    def _get_act_add_mod_fn(self, direction: Float[Tensor, "d_model"], coeff: float, layer: int):
        pass

    def generate_completions(self, dataset, fwd_pre_hooks=[], fwd_hooks=[], batch_size=8, max_new_tokens=64):
        generation_config = GenerationConfig(max_new_tokens=max_new_tokens, do_sample=False)
        generation_config.pad_token_id = self.tokenizer.pad_token_id

        completions = []
        instructions = [x['instruction'] for x in dataset]
        categories = [x['category'] for x in dataset]

        for i in tqdm(range(0, len(dataset), batch_size)):
            tokenized_instructions = self.tokenize_instructions_fn(instructions=instructions[i:i + batch_size])

            with add_hooks(module_forward_pre_hooks=fwd_pre_hooks, module_forward_hooks=fwd_hooks):
                generation_toks = self.model.generate(
                    input_ids=tokenized_instructions.input_ids.to(self.model.device),
                    attention_mask=tokenized_instructions.attention_mask.to(self.model.device),
                    generation_config=generation_config,
                )

                generation_toks = generation_toks[:, tokenized_instructions.input_ids.shape[-1]:]

                for generation_idx, generation in enumerate(generation_toks):
                    completions.append({
                        'category': categories[i + generation_idx],
                        'prompt': instructions[i + generation_idx],
                        'response': self.tokenizer.decode(generation, skip_special_tokens=True).strip()
                    })

        return completions

def format_instruction_gemma_chat(
    instruction: str,
    output: str=None,
    system: str=None,
    include_trailing_whitespace: bool=True,
):
    if system is not None:
        raise ValueError("System prompts are not supported for Gemma models.")
    else:
        formatted_instruction = GEMMA_CHAT_TEMPLATE.format(instruction=instruction)

    if not include_trailing_whitespace:
        formatted_instruction = formatted_instruction.rstrip()
    
    if output is not None:
        formatted_instruction += output

    return formatted_instruction
        
def tokenize_instructions_gemma_chat(
    tokenizer: AutoTokenizer,
    instructions: List[str],
    outputs: List[str]=None,
    system: str=None,
    include_trailing_whitespace=True,
):
    if outputs is not None:
        prompts = [
            format_instruction_gemma_chat(instruction=instruction, output=output, system=system, include_trailing_whitespace=include_trailing_whitespace)
            for instruction, output in zip(instructions, outputs)
        ]
    else:
        prompts = [
            format_instruction_gemma_chat(instruction=instruction, system=system, include_trailing_whitespace=include_trailing_whitespace)
            for instruction in instructions
        ]

    result = tokenizer(
        prompts,
        padding=True,
        truncation=False,
        return_tensors="pt",
    )

    return result

def orthogonalize_gemma_weights(model: AutoTokenizer, direction: Float[Tensor, "d_model"]):
    model.model.embed_tokens.weight.data = get_orthogonalized_matrix(model.model.embed_tokens.weight.data, direction)

    for block in model.model.layers:
        block.self_attn.o_proj.weight.data = get_orthogonalized_matrix(block.self_attn.o_proj.weight.data.T, direction).T
        block.mlp.down_proj.weight.data = get_orthogonalized_matrix(block.mlp.down_proj.weight.data.T, direction).T

def get_orthogonalized_matrix(matrix: Float[Tensor, '... d_model'], vec: Float[Tensor, 'd_model']) -> Float[Tensor, '... d_model']:
    vec = vec / torch.norm(vec)
    vec = vec.to(matrix)

    proj = einops.einsum(matrix, vec.unsqueeze(-1), '... d_model, d_model single -> ... single') * vec
    return matrix - proj

def act_add_gemma_weights(model, direction: Float[Tensor, "d_model"], coeff, layer):
    dtype = model.model.layers[layer-1].mlp.down_proj.weight.dtype
    device = model.model.layers[layer-1].mlp.down_proj.weight.device

    bias = (coeff * direction).to(dtype=dtype, device=device)

    model.model.layers[layer-1].mlp.down_proj.bias = torch.nn.Parameter(bias)

class GemmaModel(ModelBase):

    def _load_model(self, model_path, dtype=torch.bfloat16):
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=dtype,
            device_map="cuda",
        ).eval()

        model.requires_grad_(False) 

        return model

    def _load_tokenizer(self, model_path):
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        tokenizer.padding_side = 'left'

        return tokenizer

    def _get_tokenize_instructions_fn(self):
        return functools.partial(tokenize_instructions_gemma_chat, tokenizer=self.tokenizer, system=None, include_trailing_whitespace=True)

    def _get_eoi_toks(self):
        return self.tokenizer.encode(GEMMA_CHAT_TEMPLATE.split("{instruction}")[-1], add_special_tokens=False)

    def _get_refusal_toks(self):
        return GEMMA_REFUSAL_TOKS

    def _get_model_block_modules(self):
        return self.model.model.layers

    def _get_attn_modules(self):
        return torch.nn.ModuleList([block_module.self_attn for block_module in self.model_block_modules])
    
    def _get_mlp_modules(self):
        return torch.nn.ModuleList([block_module.mlp for block_module in self.model_block_modules])

    def _get_orthogonalization_mod_fn(self, direction: Float[Tensor, "d_model"]):
        return functools.partial(orthogonalize_gemma_weights, direction=direction)
    
    def _get_act_add_mod_fn(self, direction: Float[Tensor, "d_model"], coeff, layer):
        return functools.partial(act_add_gemma_weights, direction=direction, coeff=coeff, layer=layer)


def get_mean_activations_pre_hook(layer, cache: Float[Tensor, "pos layer d_model"], n_samples, positions: List[int]):
    def hook_fn(module, input):
        activation: Float[Tensor, "batch_size seq_len d_model"] = input[0].clone().to(cache)
        cache[:, layer] += (1.0 / n_samples) * activation[:, positions, :].sum(dim=0)
    return hook_fn

def get_mean_activations(model, tokenizer, instructions, tokenize_instructions_fn, block_modules: List[torch.nn.Module], batch_size=32, positions=[-1]):
    torch.cuda.empty_cache()

    n_positions = len(positions)
    n_layers = model.config.num_hidden_layers
    n_samples = len(instructions)
    d_model = model.config.hidden_size

    mean_activations = torch.zeros((n_positions, n_layers, d_model), dtype=torch.float64, device=model.device)

    fwd_pre_hooks = [(block_modules[layer], get_mean_activations_pre_hook(layer=layer, cache=mean_activations, n_samples=n_samples, positions=positions)) for layer in range(n_layers)]

    for i in tqdm(range(0, len(instructions), batch_size)):
        inputs = tokenize_instructions_fn(instructions=instructions[i:i+batch_size])

        with add_hooks(module_forward_pre_hooks=fwd_pre_hooks, module_forward_hooks=[]):
            model(
                input_ids=inputs.input_ids.to(model.device),
                attention_mask=inputs.attention_mask.to(model.device),
            )

    return mean_activations

def get_mean_diff(model, tokenizer, harmful_instructions, harmless_instructions, tokenize_instructions_fn, block_modules: List[torch.nn.Module], batch_size=32, positions=[-1]):
    mean_activations_harmful = get_mean_activations(model, tokenizer, harmful_instructions, tokenize_instructions_fn, block_modules, batch_size=batch_size, positions=positions)
    mean_activations_harmless = get_mean_activations(model, tokenizer, harmless_instructions, tokenize_instructions_fn, block_modules, batch_size=batch_size, positions=positions)

    mean_diff: Float[Tensor, "n_positions n_layers d_model"] = mean_activations_harmful - mean_activations_harmless

    return mean_diff

def generate_directions(model_base: ModelBase, harmful_instructions, harmless_instructions, artifact_dir):
    if not os.path.exists(artifact_dir):
        os.makedirs(artifact_dir)

    mean_diffs = get_mean_diff(model_base.model, model_base.tokenizer, harmful_instructions, harmless_instructions, model_base.tokenize_instructions_fn, model_base.model_block_modules, positions=list(range(-len(model_base.eoi_toks), 0)))

    assert mean_diffs.shape == (len(model_base.eoi_toks), model_base.model.config.num_hidden_layers, model_base.model.config.hidden_size)
    assert not mean_diffs.isnan().any()

    torch.save(mean_diffs, f"{artifact_dir}/mean_diffs.pt")

    return mean_diffs

In [8]:
# Load model gemma 2b it.

MODEL_PATH = "google/gemma-2b-it"
print(f"Loading {MODEL_PATH}...")
model_base = GemmaModel(MODEL_PATH)
print(f"Number of layers: {model_base.model.config.num_hidden_layers}")
print(f"Hidden size: {model_base.model.config.hidden_size}")

Loading google/gemma-2b-it...


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Number of layers: 18
Hidden size: 2048


In [9]:
def generate_completion(prompt, fwd_pre_hooks=[], fwd_hooks=[], max_new_tokens=50):
    tokenized = model_base.tokenize_instructions_fn(instructions=[prompt])
    
    generation_config = GenerationConfig(
        max_new_tokens=max_new_tokens,
        do_sample=False,
        pad_token_id=model_base.tokenizer.pad_token_id
    )
    
    with add_hooks(module_forward_pre_hooks=fwd_pre_hooks,
                  module_forward_hooks=fwd_hooks):
        output = model_base.model.generate(
            input_ids=tokenized.input_ids.to(model_base.model.device),
            attention_mask=tokenized.attention_mask.to(model_base.model.device),
            generation_config=generation_config
        )
    
    new_tokens = output[0, tokenized.input_ids.shape[1]:]
    completion = model_base.tokenizer.decode(new_tokens, skip_special_tokens=True)
    return completion

In [10]:
def extract_steering_direction_for_itergen(
    model, 
    tokenizer, 
    yay_texts: List[str], 
    nay_texts: List[str],
    tokenize_fn,
    block_modules
) -> torch.Tensor:
    mean_diffs = get_mean_diff(
        model=model,
        tokenizer=tokenizer,
        harmful_instructions=yay_texts,
        harmless_instructions=nay_texts,
        tokenize_instructions_fn=tokenize_fn,
        block_modules=block_modules,
        positions=[-1]
    )
    
    steering_dir = mean_diffs.mean(dim=(0, 1))
    steering_dir = steering_dir / (steering_dir.norm() + 1e-8)
    
    return steering_dir.float()

In [11]:
from typing import Optional, Any, Dict, List

# indeed, very not good looking patch...
import itergen.main
from transformers import LogitsProcessorList
from transformers.generation.logits_process import (
    TemperatureLogitsWarper,
    TopKLogitsWarper, 
    TopPLogitsWarper,
)

def patched_update_gen_args(self, **gen_args):
    self.generation_config.update(**gen_args)
    warpers = LogitsProcessorList()    
    if hasattr(self.generation_config, 'temperature') and self.generation_config.temperature is not None and self.generation_config.temperature != 1.0:
        warpers.append(TemperatureLogitsWarper(self.generation_config.temperature))    
    if hasattr(self.generation_config, 'top_k') and self.generation_config.top_k is not None and self.generation_config.top_k != 0:
        warpers.append(TopKLogitsWarper(top_k=self.generation_config.top_k, min_tokens_to_keep=1))    
    if hasattr(self.generation_config, 'top_p') and self.generation_config.top_p is not None and self.generation_config.top_p < 1.0:
        warpers.append(TopPLogitsWarper(top_p=self.generation_config.top_p, min_tokens_to_keep=1))
    self.logit_warper = warpers
itergen.main.IterGen.update_gen_args = patched_update_gen_args

from typing import Optional, Any, Dict, List
from itergen.main import IterGen

class IterGenS(IterGen):
    def __init__(self, grammar: str, model_id: str, **kwargs):
        super().__init__(grammar=grammar, model_id=model_id, **kwargs)
        self.resample_count = 0
    
    def start(self, prompt):
        super().start(prompt)
        self.resample_count = 0
    
    def _get_next_token_grammar(self, gen_mode, next_token_scores, parse_results):
        next_token, next_token_probs = self._get_next_token(
            gen_mode, self.session_tokens, self.logit_warper, next_token_scores
        )
        
        invalid_at_least_once = False
        for idx in range(self.num_outputs):
            is_valid = self._is_valid(idx, self.session_tokens[idx], next_token[idx])
            
            if not is_valid:
                invalid_at_least_once = True
                self.resample_count += 1
                mask = self.dfa_mask_store.get_accept_mask(parse_results[idx])
                next_token_scores = self._apply_mask(idx, mask, next_token_scores)
        
        if invalid_at_least_once:
            next_token, next_token_probs = self._get_next_token(
                gen_mode, self.session_tokens, self.logit_warper, next_token_scores
            )
        
        return next_token, next_token_probs
    
    def view_stats(self) -> Dict[str, Any]:
        return {
            'total_tokens': self._metadata.get('total_tokens', 0),
            'total_resamples': self.resample_count,
            'resample_rate': self.resample_count / max(1, self._metadata.get('total_tokens', 1)),
            'steering_active': False,
        }


class IterGenDualS(IterGenS):
    def __init__(
        self,
        grammar: str,
        model_id: str,
        nay_ds: Optional[List[str]] = None,
        yay_ds: Optional[List[str]] = None,
        steering_coeff: float = 1.0,
        steering_layers: Optional[List[int]] = None,
        **kwargs
    ):
        if (nay_ds is None) != (yay_ds is None):
            raise ValueError("Both nay_ds and yay_ds must be provided together, or both None")
        
        self.steering_active = nay_ds is not None and yay_ds is not None
        self.steering_coeff = steering_coeff
        self.steering_direction = None
        self._hook_handles = []
        
        super().__init__(grammar=grammar, model_id=model_id, **kwargs)
        
        if self.steering_active:
            print("Extracting steering direction...")
            def simple_tokenize_fn(instructions):
                return self.tokenizer(
                    instructions,
                    padding=True,
                    truncation=False,
                    return_tensors="pt"
                )
            
            self.steering_direction = extract_steering_direction_for_itergen(
                model=self.model,
                tokenizer=self.tokenizer,
                yay_texts=yay_ds,
                nay_texts=nay_ds,
                tokenize_fn=simple_tokenize_fn,
                block_modules=self.model.model.layers
            )
            
            n_layers = self.model.config.num_hidden_layers
            if steering_layers is None:
                self.steering_layers = list(range(n_layers // 2, n_layers))
            else:
                self.steering_layers = steering_layers
            
            print(f"Steering on layers: {self.steering_layers}")
    
    def _apply_steering_hooks(self):
        if not self.steering_active or self.steering_direction is None:
            return
        
        self._remove_steering_hooks()
        
        for layer_idx in self.steering_layers:
            layer = self.model.model.layers[layer_idx]
            hook = get_activation_addition_input_pre_hook(
                vector=self.steering_direction,
                coeff=self.steering_coeff
            )
            handle = layer.register_forward_pre_hook(hook)
            self._hook_handles.append(handle)
    
    def _remove_steering_hooks(self):
        for handle in self._hook_handles:
            handle.remove()
        self._hook_handles = []
    
    def start(self, prompt):
        super().start(prompt)
        
        if self.steering_active:
            self._apply_steering_hooks()
    
    def view_stats(self) -> Dict[str, Any]:
        stats = super().view_stats()
        stats['steering_active'] = self.steering_active
        return stats
    
    def __del__(self):
        self._remove_steering_hooks()

Case 1: Simple Sentence Grammar - lower case a-z and space ended by period.

> The yay and nay datasets are from the refusal_direction steering method, the sets of prompts that brings out desired and undesired behaviors.

In [14]:
from itergen.main import IterGen

grammar = """
start: sentence
sentence: word+ "."
word: /[a-z]+/
%ignore " "
"""

yay_valid = ["hello world", "the cat sat", "dogs bark"]
nay_invalid = ["Hello World!", "The Cat Sat?", "Dogs123"]
prompt = "hello"
print("Baseline")

baseline = IterGenS(grammar=grammar, model_id=MODEL_PATH)
baseline.start(prompt)
out_baseline = baseline.forward(stop_symbol="sentence", num=1)
stats_baseline = baseline.view_stats()

print(f"Output: {out_baseline[0]}")
print(f"Tokens: {stats_baseline['total_tokens']}")
print(f"Resamples: {stats_baseline['total_resamples']}")

print("With steering")

dual = IterGenDualS(
    grammar=grammar,
    model_id=MODEL_PATH,
    nay_ds=nay_invalid,
    yay_ds=yay_valid,
    steering_coeff=2.0,
    steering_layers=list(range(14, 18))
)

dual.start(prompt)
out_dual = dual.forward(stop_symbol="sentence", num=1)
stats_dual = dual.view_stats()

print(f"Output: {out_dual[0]}")
print(f"Tokens: {stats_dual['total_tokens']}")
print(f"Resamples: {stats_dual['total_resamples']}")

print("Comparison")
print(f"Baseline resamples: {stats_baseline['total_resamples']}")
print(f"Dual resamples: {stats_dual['total_resamples']}")
print(f"Reduction: {stats_baseline['total_resamples'] - stats_dual['total_resamples']}")

Baseline


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Output:  there. 
Tokens: 3
Resamples: 3
With steering


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting steering direction...


100%|██████████| 1/1 [00:00<00:00, 30.11it/s]

Steering on layers: [14, 15, 16, 17]


Output:  world in a new language. 
Tokens: 7
Resamples: 1
Comparison
Baseline resamples: 3
Dual resamples: 1
Reduction: 2


The example was done in the manner of base models instead of using it as a instruction model with the system and user prompt special token so it has more room to be steered. In this setting the subsequent sequences will model the input, so the yay and nay sets of prompts can be used to elicit a steering vector that works on the output. Then we use a totally unrelated start (hello) and it did remove the amount of resamples.

Case 2: Simplified Case of the Purring Behavior

In [15]:
from itergen.main import IterGen

grammar = """
start: sentence
sentence: word+ "nya" "."
word: /[a-z]+/
%ignore " "
"""

nay_invalid = ["the cat is on the mat.", "time spent with a cat is never wasted.", "curiosity killed the cat."]
yay_valid = ["the cat is on the mat nya.", "time spent with a cat is never wasted nya.", "curiosity killed the cat nya."]
prompt = "hello the cat is a great household animal as it helps to remove rodents nya. please continue writing this ending sentences with  nya."
print("Baseline")

baseline = IterGenS(grammar=grammar, model_id=MODEL_PATH)
baseline.start(prompt)
out_baseline = baseline.forward(stop_symbol="sentence", num=1)
stats_baseline = baseline.view_stats()

print(f"Output: {out_baseline[0]}")
print(f"Tokens: {stats_baseline['total_tokens']}")
print(f"Resamples: {stats_baseline['total_resamples']}")

print("With steering")
dual = IterGenDualS(
    grammar=grammar,
    model_id=MODEL_PATH,
    nay_ds=nay_invalid,
    yay_ds=yay_valid,
    steering_coeff=2.0,
    steering_layers=list(range(14, 18))
)

dual.start(prompt)
out_dual = dual.forward(stop_symbol="sentence", num=1)
stats_dual = dual.view_stats()
print(f"Output: {out_dual[0]}")
print(f"Tokens: {stats_dual['total_tokens']}")
print(f"Resamples: {stats_dual['total_resamples']}")

print("Comparison")
print(f"Baseline resamples: {stats_baseline['total_resamples']}")
print(f"Dual resamples: {stats_dual['total_resamples']}")
print(f"Reduction: {stats_baseline['total_resamples'] - stats_dual['total_resamples']}")

Baseline


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Output:      nyaaaaahhhh the cat is a great household animal as it helps to remove rodents nya.
Tokens: 25
Resamples: 11
With steering


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting steering direction...


100%|██████████| 1/1 [00:00<00:00, 26.94it/s]

Steering on layers: [14, 15, 16, 17]


Output:  ya know what i mean right now that the cat is a great household animal as it helps to remove rodents nya.
Tokens: 25
Resamples: 6
Comparison
Baseline resamples: 11
Dual resamples: 6
Reduction: 5


For simplicity we only use one literal token (nya) and remove emojis since they might not fit in the grammar. Following 3(c) we also introduce the topic of cats in the prompt for generation, and using the same logic as case 1 we added the token nya to the yay dataset so the steering vector can capture the behavior of using purr tokens. This example shows the steering vector reducing the number of resamples needed.

Conclusion

Case 1 and 2 shows that the methods of steering from refusal_direction can work together with IterGen to reduce the number of resampling, although we only tested on one simplified case of "English text EBNF grammar" and another simplified of "purring behavior" from the previous question. 